In [1]:
try:
    sc.stop()
except:
    pass

from pyspark import SparkContext, SparkConf
conf = SparkConf().setAppName('Spark-logAnalysis').setMaster('local[4]')
sc = SparkContext.getOrCreate(conf = conf)

from pyspark.sql import SparkSession 
from pyspark.sql.types import StructField, StringType, IntegerType, StructType

spark = SparkSession.builder.appName('sparklog').getOrCreate()

rdd = sc.textFile("file:///home/hadoop/data/access_log_Jul95")

rdd.count() 
rdd.take(5) 

logs_df = spark.read.text('file:///home/hadoop/data/access_log_Jul95')

logs_df.show(10)

sample_logs = [item['value'] for item in logs_df.take(15)]
sample_logs

import re 
host_pattern = r'(^\S+\.[\S+\.]+\S+)\s'

hosts = [re.search(host_pattern, item).group(1)
        if re.search(host_pattern, item)
        else 'no match'
        for item in sample_logs]


ts_pattern = r'\[(\d{2}/\w{3}/\d{4}:\d{2}:\d{2}:\d{2} -\d{4})]'
timestamps = [re.search(ts_pattern, item).group(1) for item in sample_logs]
timestamps

method_uri_pattern = r'\"(\S+)\s(\S+)\s*(\S*)\"'
method_uri_protocol = [re.search(method_uri_pattern, item).groups() 
                       if re.search(method_uri_pattern, item)
                       else 'no match'
                       for item in sample_logs]
method_uri_protocol

status_pattern = r'\s(\d{3})\s'
# $ This the end of line
content_size_pattern = r'\s(\d+)$'


from pyspark.sql.functions import regexp_extract

logfile_df = logs_df.select(regexp_extract('value', host_pattern, 1).alias('host'), 
                     regexp_extract('value', ts_pattern, 1).alias('timestamp'),
                     regexp_extract('value', method_uri_pattern, 1).alias('method'),
                     regexp_extract('value', method_uri_pattern, 2).alias('endpoint'),
                     regexp_extract('value', method_uri_pattern, 3).alias('protocol'), 
                     regexp_extract('value', status_pattern, 1).alias('status'), 
                     regexp_extract('value', content_size_pattern, 1).cast('integer').alias('content_size'))
logfile_df.show(10) 

logfile_df.filter(logfile_df['host'].isNull()).count()

logfile_df.filter(logfile_df['content_size'].isNull()).count()


# fill missing values with 0
logfile_df = logfile_df.na.fill({'content_size' : 0}) 


logfile_df.filter(logfile_df['content_size'].isNull()).count() 


from pyspark.sql.functions import col
from pyspark.sql.functions import sum as spark_sum

def count_na(col_name):
    return spark_sum(col(col_name).isNull().cast('integer')).alias(col_name)

# Build a list of column expressions , one per column
exprs = [count_na(col_name) for col_name in logfile_df.columns]

# *exprs converts the list of expression into variable function arguments i.e. unpacks the list
logfile_df.agg(*exprs).show()


# RDD - persistance & cache
# cache - IN Memory
# pesist - DISK_ONLY, MEMORY_AND_DISK, MEMORY_ONLY
logfile_df.cache()

%%time
logfile_df.printSchema()


+--------------------+
|               value|
+--------------------+
|199.72.81.55 - - ...|
|unicomp6.unicomp....|
|199.120.110.21 - ...|
|burger.letters.co...|
|199.120.110.21 - ...|
|burger.letters.co...|
|burger.letters.co...|
|205.212.115.106 -...|
|d104.aa.net - - [...|
|129.94.144.152 - ...|
+--------------------+
only showing top 10 rows

+--------------------+--------------------+------+--------------------+--------+------+------------+
|                host|           timestamp|method|            endpoint|protocol|status|content_size|
+--------------------+--------------------+------+--------------------+--------+------+------------+
|        199.72.81.55|01/Jul/1995:00:00...|   GET|    /history/apollo/|HTTP/1.0|   200|        6245|
|unicomp6.unicomp.net|01/Jul/1995:00:00...|   GET| /shuttle/countdown/|HTTP/1.0|   200|        3985|
|      199.120.110.21|01/Jul/1995:00:00...|   GET|/shuttle/missions...|HTTP/1.0|   200|        4085|
|  burger.letters.com|01/Jul/1995:00:00...|   

UsageError: Line magic function `%%time` not found.


In [5]:

logfile_df.describe(['content_size']).toPandas()

status_code_df = (logfile_df.groupby('status').count().sort('status').cache())

status_code_df.count() 

status_code_final = status_code_df.toPandas().sort_values(by = ['count'], ascending=False)
status_code_final 

import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd 

sns.catplot(x = 'status', y = 'count', data = status_code_final , kind = 'bar', height=8) 

from pyspark.sql.functions import log

log_freq_df = status_code_df.withColumn('log(count)', log(status_code_df['count']))
log_freq_df.show()

log_freq_final = log_freq_df.toPandas().sort_values(by = ['log(count)'], ascending=False)






sns.catplot(x = 'status', y = 'log(count)', data = log_freq_final , kind = 'bar', height=8)


+------+-------+------------------+
|status|  count|        log(count)|
+------+-------+------------------+
|      |      1|               0.0|
|   200|1701536|14.347041930501215|
|   302|  46573| 10.74877625305556|
|   304| 132627|11.795295955915682|
|   400|      5|1.6094379124341003|
|   403|     54|3.9889840465642745|
|   404|  10843| 9.291274989472182|
|   500|     62| 4.127134385045092|
|   501|     14|2.6390573296152584|
+------+-------+------------------+



In [3]:
unique_host_count = logfile_df.select('host').distinct().count()

In [4]:
unique_host_count

81961

In [6]:
from pyspark.sql import functions as F

In [7]:
host_day_df = logfile_df.select(logfile_df.host,
                               F.dayofmonth('timestamp').alias('day'))
host_day_df.show(5)

+--------------------+----+
|                host| day|
+--------------------+----+
|        199.72.81.55|null|
|unicomp6.unicomp.net|null|
|      199.120.110.21|null|
|  burger.letters.com|null|
|      199.120.110.21|null|
+--------------------+----+
only showing top 5 rows



In [8]:
logfile_df.select('timestamp').show(5)

+--------------------+
|           timestamp|
+--------------------+
|01/Jul/1995:00:00...|
|01/Jul/1995:00:00...|
|01/Jul/1995:00:00...|
|01/Jul/1995:00:00...|
|01/Jul/1995:00:00...|
+--------------------+
only showing top 5 rows



In [9]:
from pyspark.sql.functions import udf

month_map = {
  'Jan': 1, 'Feb': 2, 'Mar':3, 'Apr':4, 'May':5, 'Jun':6, 'Jul':7,
  'Aug':8,  'Sep': 9, 'Oct':10, 'Nov': 11, 'Dec': 12
}

def parse_clf_time(text):
    return "{0:04d}-{1:02d}-{2:02d} {3:02d}:{4:02d}:{5:02d}".format(
      int(text[7:11]),
      month_map[text[3:6]],
      int(text[0:2]),
      int(text[12:14]),
      int(text[15:17]),
      int(text[18:20])
    )

In [10]:
udf_time = udf(parse_clf_time)

logfile_df = logfile_df.select('*', udf_time(logfile_df['timestamp']).cast('timestamp').alias('time')).drop('timestamp')



In [11]:
logfile_df.show(10)

+--------------------+------+--------------------+--------+------+------------+-------------------+
|                host|method|            endpoint|protocol|status|content_size|               time|
+--------------------+------+--------------------+--------+------+------------+-------------------+
|        199.72.81.55|   GET|    /history/apollo/|HTTP/1.0|   200|        6245|1995-07-01 00:00:01|
|unicomp6.unicomp.net|   GET| /shuttle/countdown/|HTTP/1.0|   200|        3985|1995-07-01 00:00:06|
|      199.120.110.21|   GET|/shuttle/missions...|HTTP/1.0|   200|        4085|1995-07-01 00:00:09|
|  burger.letters.com|   GET|/shuttle/countdow...|HTTP/1.0|   304|           0|1995-07-01 00:00:11|
|      199.120.110.21|   GET|/shuttle/missions...|HTTP/1.0|   200|        4179|1995-07-01 00:00:11|
|  burger.letters.com|   GET|/images/NASA-logo...|HTTP/1.0|   304|           0|1995-07-01 00:00:12|
|  burger.letters.com|   GET|/shuttle/countdow...|HTTP/1.0|   200|           0|1995-07-01 00:00:12|


In [12]:
host_day_df = logfile_df.select(logfile_df.host,
                               F.dayofmonth('time').alias('day'))
host_day_df.show(5)

+--------------------+---+
|                host|day|
+--------------------+---+
|        199.72.81.55|  1|
|unicomp6.unicomp.net|  1|
|      199.120.110.21|  1|
|  burger.letters.com|  1|
|      199.120.110.21|  1|
+--------------------+---+
only showing top 5 rows



In [13]:
host_day_distinct_df = host_day_df.dropDuplicates()

host_day_distinct_df.show(5)



Py4JJavaError: An error occurred while calling o266.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 3 in stage 43.0 failed 1 times, most recent failure: Lost task 3.0 in stage 43.0 (TID 686, localhost, executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/usr/local/spark/python/lib/pyspark.zip/pyspark/worker.py", line 377, in main
    process()
  File "/usr/local/spark/python/lib/pyspark.zip/pyspark/worker.py", line 372, in process
    serializer.dump_stream(func(split_index, iterator), outfile)
  File "/usr/local/spark/python/lib/pyspark.zip/pyspark/serializers.py", line 352, in dump_stream
    self.serializer.dump_stream(self._batched(iterator), stream)
  File "/usr/local/spark/python/lib/pyspark.zip/pyspark/serializers.py", line 142, in dump_stream
    for obj in iterator:
  File "/usr/local/spark/python/lib/pyspark.zip/pyspark/serializers.py", line 341, in _batched
    for item in iterator:
 

In [15]:
host_day_distinct_df.take(5)


Py4JJavaError: An error occurred while calling o315.collectToPython.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 3 in stage 45.0 failed 1 times, most recent failure: Lost task 3.0 in stage 45.0 (TID 690, localhost, executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/usr/local/spark/python/lib/pyspark.zip/pyspark/worker.py", line 377, in main
    process()
  File "/usr/local/spark/python/lib/pyspark.zip/pyspark/worker.py", line 372, in process
    serializer.dump_stream(func(split_index, iterator), outfile)
  File "/usr/local/spark/python/lib/pyspark.zip/pyspark/serializers.py", line 352, in dump_stream
    self.serializer.dump_stream(self._batched(iterator), stream)
  File "/usr/local/spark/python/lib/pyspark.zip/pyspark/serializers.py", line 142, in dump_stream
    for obj in iterator:
  File "/usr/local/spark/python/lib/pyspark.zip/pyspark/serializers.py", line 341, in _batched
    for item in iterat

In [ ]:
host_day_df.groupby('day').count().sort('day').toPandas()

In [16]:
sc.stop()